# Introduction

This is the first step which generates the data from the datacube.   
We were provided of the linescan id numbers by which we can get the linescan images.   
When the linescan image data is retrieved, we find the minimum and maximum of the longitude and the latitude, and find the median color values between 2017 and 2019 from the same coordinate.   
After finding the median, we will add two variables; the NDVI and the MNDWI index.

When everything is ready, we also engineered 25 different masks using a different combination of maskOpen and maskClose parameters.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datacube import Datacube
import matplotlib.pyplot as plt
import sys
sys.path.append("../scripts")
from datacube.utils.cog import write_cog
from dea_bandindices import calculate_indices
from dea_datahandling import load_ard
from dea_dask import create_local_dask_cluster
from mpl_toolkits.mplot3d import Axes3D #to plot in 3D
import matplotlib.pyplot as plt         #to plot arrays as images
from matplotlib import colors           #extra colour representations
from matplotlib import cm               #extra colour representations
import numpy as np                      #to manipulate arrays
import cv2                              #to perform more complex manipulations on arrays
import geopandas as gpd

from dea_plotting import map_shapefile
from dea_spatialtools import xr_rasterize
import xarray as xr

/env/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
create_local_dask_cluster()
dc = Datacube(app = "")

Client Scheduler: tcp://127.0.0.1:37821 Dashboard: /proxy/8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.95 GB


/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,


In [3]:
linescan_datasets = dc.find_datasets(product='linescan')
linescan_datasets = sorted(linescan_datasets, key = lambda ds: (ds.center_time, ds.id))
linescan_datasets = pd.Series(linescan_datasets)
indices = pd.read_csv('../03_EY_challenge1/trainindices.csv')

In [4]:
indices.head()

,ID,DD,gdf,train
0,I1,0.0,ABERFELDY_WEST_200_P1_201901260955_MGA94_55,23
1,I2,1.0,ABERFELDY_WEST_214_P1_201901261750_MGA94_55,26
2,I3,2.0,CREAM_JIM_JORDAN_217_P1_201901262218_MGA94_55,27
3,I4,5.0,JORDAN_231_P1_201901271500_MGA94_55,29
4,I5,6.0,JORDAN_234_P1_201901271901_MGA94_55,31


In [5]:
vector_file = '../03_EY_challenge1/resources/fire_boundaries.shp'
gdf = gpd.read_file(vector_file)
def clean_name(name):
    if name is None:
        res = None
    else:
        if name.upper()[-4::] == ".JPG":
            res = name.upper()[:-4].replace(' ','_')
        else:
            res = name.upper().replace(' ','_')
    return res
gdf['SourceNameClean'] = gdf.apply(lambda row: clean_name(row.SourceName), axis=1)
gdf.dtUTC = gdf.apply(lambda row: datetime.strptime(row.dtUTC, '%Y-%m-%d %H:%M:%S'), axis=1)
gdf.dtLocal = gdf.apply(lambda row: datetime.strptime(row.dtLocal, '%Y-%m-%d %H:%M:%S'), axis=1)

In [6]:
open_array = np.array([1,3,5,7,9])
close_array = np.array([25,36, 49, 64,81]) 
comb = []
for o in open_array:
    for c in close_array:
        comb.append((int(o),int(c)))

Throught the for-loop process below, we will retrieve the linescan image data and aquire their coordinate information. Using the cooridnate information, we will obtain median of NBART colors (blue, green, red, nir, swir_2, swir_1), and will also compute the NDVI and MNDWI.   

Finally, we first select the pixels whose linesan values are greater than 100. Then, we will engineer 25 different masks using the combination of the parameters for the maskOpen and maskClose method. When they are generated, having the value 255 means they are inside the given mask and having 0 means they are outside of the mask. The initial assumption was that these binary variables would be useful for the tree-based models. 

Below is the combination of the parameters.

In [9]:
pd.Series(comb)

0     (1, 25)
1     (1, 36)
2     (1, 49)
3     (1, 64)
4     (1, 81)
5     (3, 25)
6     (3, 36)
7     (3, 49)
8     (3, 64)
9     (3, 81)
10    (5, 25)
11    (5, 36)
12    (5, 49)
13    (5, 64)
14    (5, 81)
15    (7, 25)
16    (7, 36)
17    (7, 49)
18    (7, 64)
19    (7, 81)
20    (9, 25)
21    (9, 36)
22    (9, 49)
23    (9, 64)
24    (9, 81)
dtype: object

Below is the for-loop process explained earlier.

In [ ]:
for index, gdf_name in zip(indices.train, indices.gdf):
    dc = Datacube(app = "")
    src = dc.load(product='linescan', id=linescan_datasets[index].id, output_crs='epsg:28355', resolution=(-10,10))
    ob = gdf.loc[gdf.SourceNameClean == gdf_name]
    src['tgt'] = xr_rasterize(gdf=ob, da=src)
    src = src.isel(time = 0)
    crs = src.crs
    x_min, x_max, y_min, y_max = int(src.x.min().values), int(src.x.max().values), int(src.y.min().values), int(src.y.max().values)
    lon_range = (x_min, x_max)
    lat_range = (y_min, y_max)
    resolution = (-10, 10)

    query = {
        'x': lon_range,
        'y': lat_range,
        'time': ('2017','2019'),
        'measurements': [
            'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_2',
            'nbart_swir_1'
        ],
        'crs' : crs,
        'output_crs': crs,
        'resolution': (-10, 10),
        'group_by': 'solar_day',
    }
    history = load_ard(dc=dc, products=['ga_ls8c_ard_3','ga_ls7e_ard_3'], 
                              dask_chunks={'time': 1}, min_gooddata=0.85, ls7_slc_off=True, **query) 
    history = calculate_indices(history, index=['NDVI','MNDWI'], collection='ga_ls_3').median('time') 
    src = src.merge(history)
    del history

    lowerBound=100
    upperBound=255

    #create the mask over the top of our original image
    linescan_mask = cv2.inRange(src.linescan.values,lowerBound,upperBound)

    for i, (o, c) in enumerate(comb):

        #establish dialation and contraction parameters
        kernelOpen=np.ones((o,o)) # try it yourself!
        kernelClose=np.ones((c,c))    

        #denoise the pixels
        maskOpen=cv2.morphologyEx(linescan_mask,cv2.MORPH_OPEN,kernelOpen)

        maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

        src[f'maskOpen_{i}'] = (['y','x'], maskOpen)
        src[f'maskClose_{i}'] = (['y','x'], maskClose)
        
        del maskOpen
        del maskClose
    df = src.to_dataframe()
    del src
    del dc
    df = df.reset_index().drop(['time', 'spatial_ref'], axis = 1)
    tgt = df.tgt
    df.drop('tgt', axis = 1, inplace = True)
    df['tgt'] = tgt
    del tgt
    df.to_csv(f'./Datasets/train_{index}.csv', index = False)
    del df

## Below is the same process for the test images

In [10]:
test = pd.read_csv('../03_EY_challenge1/test.csv')
fname = test.label.unique()

In [11]:
fname

array(['JORDAN_235_P1_201901281204_MGA94_55',
       'JORDAN_294_P1_201902011150_MGA94_55',
       'WALHALLA_313_P1_201902020733_MGA94_55',
       'WALHALLA_353_P1_201902031625_MGA94_55',
       'MACALISTER91_648_P1_201903070444_MGA94_55'], dtype=object)

In [ ]:
for j, f in enumerate(fname):
    dc = Datacube(app = "")
    src = dc.load(product='linescan', label=f, output_crs='epsg:28355', resolution=(-10,10))
    src = src.isel(time = 0)
    crs = src.crs
    x_min, x_max, y_min, y_max = int(src.x.min().values), int(src.x.max().values), int(src.y.min().values), int(src.y.max().values)
    lon_range = (x_min, x_max)
    lat_range = (y_min, y_max)
    resolution = (-10, 10)

    query = {
        'x': lon_range,
        'y': lat_range,
        'time': ('2017','2019'),
        'measurements': [
            'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_2',
            'nbart_swir_1'
        ],
        'crs' : crs,
        'output_crs': crs,
        'resolution': (-10, 10),
        'group_by': 'solar_day',
    }
    history = load_ard(dc=dc, products=['ga_ls8c_ard_3','ga_ls7e_ard_3'], 
                              dask_chunks={'time': 1}, min_gooddata=0.85, ls7_slc_off=True, **query) 
    history = calculate_indices(history, index=['NDVI','MNDWI'], collection='ga_ls_3').median('time') 
    src = src.merge(history)
    del history
    lowerBound=80
    upperBound=255

    #create the mask over the top of our original image
    linescan_mask = cv2.inRange(src.linescan.values,lowerBound,upperBound)

    for i, (o, c) in enumerate(comb):

        #establish dialation and contraction parameters
        kernelOpen=np.ones((o,o)) # try it yourself!
        kernelClose=np.ones((c,c))    

        #denoise the pixels
        maskOpen=cv2.morphologyEx(linescan_mask,cv2.MORPH_OPEN,kernelOpen)

        maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

        src[f'maskOpen_{i}'] = (['y','x'], maskOpen)
        src[f'maskClose_{i}'] = (['y','x'], maskClose)

        del maskOpen
        del maskClose
    df = src.to_dataframe()
    del src
    df.reset_index(inplace = True)
    df.drop(['time','spatial_ref'], axis = 1, inplace = True)
    df[['x','y']] = df[['x','y']].astype(int)
    df.to_csv(f'test_{j+2}.csv', index = False)
    del df